# <font color=Salmon>Imports</font>

In [ ]:
import pandas as pd
import sqlalchemy as sql
from pathlib import Path

## <font color=LightBlue>Load CSV Data</font>

In [ ]:
csv_path = Path('./database_start.csv')

vinyl_df = pd.read_csv(
    filepath_or_buffer = csv_path,
    sep = ',',
    na_values = '-',
    header = 0,
    index_col = False
    )

vinyl_df.head(10)

### <font color=LightGreen>Clean CSV Data</font>

In [ ]:
vinyl_df.dtypes

In [ ]:
#Change column dtypes from 'object' to 'string'
dtype_dict = {
    'Album Name': 'string',
    'Artist': 'string',
    'Publisher': 'string',
    'Format': 'category',
    'Source': 'string',
    'Genre': 'category',
    'Notes': 'string'
}

vinyl_df = vinyl_df.astype(dtype_dict)
vinyl_df.dtypes

In [ ]:
'''NOTE:
    Replaced by stipulating "na_values='-'"
    when first importing csv data
'''

#Replace hyphens w/ pd.NA values
#vinyl_df.replace('-', pd.NA, inplace=True)
#vinyl_df

## <font color=Gold>Summarize the Data</font>

In [ ]:
vinyl_df.info()

# <font color=Orange>Database</font>

## <font color=Salmon>Create DB</font>

In [ ]:
#Temporary sqlite database 
db_connection_string = 'sqlite:///vinyl_inventory.db'

#Database connection object
engine = sql.create_engine(db_connection_string)

#Create inspector to observe changes in the database
inspector = sql.inspect(engine)

In [ ]:
#Confirm temporary database created by pulling table names.
inspector.get_table_names()

## <font color=LightBlue>Create DB Tables</font>

### <font color=LightGreen>Create Unique Tables</font>

    ***Not added to sqlite database yet.***

In [ ]:
#Table of artist names
artist_list = list(vinyl_df['Artist'].unique())
artist_id = range(0, len(artist_list))

artist_table = pd.DataFrame(
    {
        'artist_id': artist_id,
        'artist': artist_list,
    }
)

print(artist_table)

In [ ]:
#Table of Years
year_list = list(vinyl_df['Year'].unique())
year_list.sort()

year_id = range(0, len(year_list))

year_table = pd.DataFrame(
    {
        'year_id': year_id,
        'year': year_list
    }
)

print(year_table)

In [ ]:
#Table of publishers
publisher_list = list(vinyl_df['Publisher'].unique())
publisher_list.sort()

publisher_id = range(0, len(publisher_list))

publisher_table = pd.DataFrame(
    {
        'publisher_id': publisher_id,
        'publisher': publisher_list
    }
)

print(publisher_table)

In [ ]:
#Table of vinyl record formats
#Different technique for adding an id column as first column
format_list = list(vinyl_df['Format'].unique())
format_table = pd.DataFrame(format_list, columns=['vinyl_format'])
format_table['format_id'] = format_table.index
format_table = format_table[['format_id', 'vinyl_format']]
format_table

In [ ]:
#Table of sources
source_list = list(vinyl_df['Source'].unique())
source_list[0] = 'salvatierra'
source_list[4] = 'forrest'
source_list[6] = 'ryan_stowe'

source_id = range(0, len(source_list))

source_table = pd.DataFrame(
    {
        'source_id': source_id,
        'source': source_list
    }
)

print(source_table)

In [ ]:
#Table of genres
genre_list = list(vinyl_df['Genre'].unique())
genre_id = range(0, len(genre_list))
genre_table = pd.DataFrame(
    {
        'genre_id': genre_id,
        'album_genre': genre_list
    }
)

print(genre_table)

### <font color=LightGreen>Add Tables to Database</font>

In [ ]:
#Add the tables to the sqlite database
artist_table.to_sql('artist', engine, index=False, if_exists='replace')
year_table.to_sql('release_year', engine, index=False, if_exists='replace')
publisher_table.to_sql('publisher', engine, index=False, if_exists='replace')
format_table.to_sql('vinyl_format', engine, index=False, if_exists='replace')
source_table.to_sql('source', engine, index=False, if_exists='replace')
genre_table.to_sql('note', engine, index=False, if_exists='replace')

In [ ]:
#Verify tables were imported
print(sql.inspect(engine).get_table_names())

### <font color=LightGreen>Normalize Primary Table</font>

In [ ]:
#Add primary key as album_id
album_id_list = list(vinyl_df.index)
vinyl_df['album_id'] = album_id_list 
vinyl_df = vinyl_df[
    [
        'album_id',
        'Album Name',
        'Artist',
        'Year',
        'Publisher',
        'Format',
        'Source',
        'Genre',
        'Notes'
    ]
]

vinyl_df

In [ ]:
artist_dict = dict(zip(artist_table['artist'], artist_table['artist_id']))
year_dict = dict(zip(year_table['year'], year_table['year_id']))
publisher_dict = dict(zip(publisher_table['publisher'], publisher_table['publisher_id']))
format_dict = dict(zip(format_table['vinyl_format'], format_table['format_id']))
source_dict = dict(zip(source_table['source'], source_table['source_id']))
genre_dict = dict(zip(genre_table['album_genre'], genre_table['genre_id']))

norm_dict = artist_dict
norm_dict.update(year_dict)
norm_dict.update(publisher_dict)
norm_dict.update(format_dict)
norm_dict.update(source_dict)
norm_dict.update(genre_dict)

In [ ]:
test_df = vinyl_df

"""
test_df.replace(artist_dict, inplace=True)
test_df.replace(year_dict, inplace=True)
test_df.replace(publisher_dict, inplace=True)
test_df.replace(format_dict, inplace=True)
test_df.replace(source_dict, inplace=True)
test_df.replace(genre_dict, inplace=True)
"""

test_df.replace(norm_dict, inplace=True)
test_df

In [ ]:
test_df.Notes.unique()

In [ ]:
#TODO:
# 1) organize column order
# 2) set indexes to id values in tables sent 'to_sql'
# 3) in notes, change all 1 values to NA